### モジュール読み込み

In [1]:
from pathlib import Path
import warnings

from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_pose_result)
from mmpose.datasets import DatasetInfo

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

import numpy as np
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
from time import time

/home/koudai/anaconda3/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/koudai/anaconda3/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


### 定数定義

In [2]:
# For model
device = 'cuda:0'
det_config = '../demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
pose_config = '../configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/crowdpose/hrnet_w32_crowdpose_256x192.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_crowdpose_256x192-960be101_20201227.pth'

# For image
img_root = Path('/home/koudai/data/sample/images/')
out_img_root = Path("/home/koudai/data/sample/results/")

### モデル読み込み

In [3]:
det_model = init_detector(
    det_config, det_checkpoint, device=device.lower())
# build the pose model from a config file and a checkpoint file
pose_model = init_pose_model(
    pose_config, pose_checkpoint, device=device.lower())

load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_crowdpose_256x192-960be101_20201227.pth


### データ読み込み

In [4]:
dataset = pose_model.cfg.data['test']['type']
dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
if dataset_info is None:
    warnings.warn(
        'Please set `dataset_info` in the config.'
        'Check https://github.com/open-mmlab/mmpose/pull/663 for details.',
        DeprecationWarning)
else:
    dataset_info = DatasetInfo(dataset_info)

bbox_thr = 0.3
kpt_thr = 0.3
radius = 4
thickness = 1
show = False
return_heatmap = False
output_layer_names = None

In [5]:
df = pd.DataFrame(columns=["dir", "name", "head.x", "head.y", "head.score", "neck.x", "neck.y", "neck.score", 
                           "Lshoulder.x", "Lshoulder.y", "Lshoulder.score", "Rshoulder.x", "Rshoulder.y", "Rshoulder.score", 
                           "latency.total", "latency.det", "latency.pose"
                           ]
                    )

In [6]:
imglist = sorted(list(img_root.glob("**/*.jpg")))

In [7]:
for i, imgpath in enumerate(tqdm(imglist)):
    start_time = time()
    # test a single image, the resulting box is (x1, y1, x2, y2)
    mmdet_results = inference_detector(det_model, imgpath)

    # keep the person class bounding boxes.
    det_cat_id = 1
    person_results = process_mmdet_results(mmdet_results, det_cat_id)
    det_time = time() - start_time

    pose_results, returned_outputs = inference_top_down_pose_model(
            pose_model,
            str(imgpath),
            person_results,
            bbox_thr=bbox_thr,
            format='xyxy',
            dataset=dataset,
            dataset_info=dataset_info,
            return_heatmap=return_heatmap,
            outputs=output_layer_names)
    total_time = time() - start_time
    pose_time = total_time - det_time

    result_kp = np.round(pose_results[0]['keypoints'][:,0:2], decimals= 0)
    result_score = np.round(pose_results[0]['keypoints'][:,2:3], decimals= 2)

    df.at[i,"dir"] = imgpath.parts[-2]
    df.at[i,"name"] = imgpath.name
    df.at[i, "head.x"] = result_kp[12,0].astype(int)
    df.at[i, "head.y"] = result_kp[12,1].astype(int)
    df.at[i, "head.score"] = result_score[12][0]
    df.at[i, "neck.x"] = result_kp[13,0].astype(int)
    df.at[i, "neck.y"] = result_kp[13,1].astype(int)
    df.at[i, "neck.score"] = result_score[13][0]
    df.at[i, "Lshoulder.x"] = result_kp[0,0].astype(int)
    df.at[i, "Lshoulder.y"] = result_kp[0,1].astype(int)
    df.at[i, "Lshoulder.score"] = result_score[0][0]
    df.at[i, "Rshoulder.x"] = result_kp[1,0].astype(int)
    df.at[i, "Rshoulder.y"] = result_kp[1,1].astype(int)
    df.at[i, "Rshoulder.score"] = result_score[1][0]
    df.at[i, "latency.total"] = total_time
    df.at[i, "latency.det"] = det_time
    df.at[i, "latency.pose"] = pose_time

    out_file = out_img_root / f'vis_{str(imgpath.name)}'
    vis_pose_result(
        pose_model,
        str(imgpath),
        pose_results,
        dataset=dataset,
        dataset_info=dataset_info,
        kpt_score_thr=kpt_thr,
        radius=radius,
        thickness=thickness,
        show=show,
        out_file=out_file)

100%|██████████| 4558/4558 [17:50<00:00,  4.26it/s]


In [8]:
df

,dir,name,head.x,head.y,head.score,neck.x,neck.y,neck.score,Lshoulder.x,Lshoulder.y,Lshoulder.score,Rshoulder.x,Rshoulder.y,Rshoulder.score,latency.total,latency.det,latency.pose
0,IMG_0017,IMG_0017_000001.jpg,945,216,0.91,935,658,0.9,1190,766,0.81,670,757,0.81,0.888566,0.795931,0.092636
1,IMG_0017,IMG_0017_000002.jpg,945,212,0.93,935,655,0.91,1186,765,0.83,673,755,0.81,0.183608,0.085677,0.09793
2,IMG_0017,IMG_0017_000003.jpg,945,220,0.91,935,655,0.91,1192,764,0.81,668,764,0.82,0.177199,0.083027,0.094172
3,IMG_0017,IMG_0017_000004.jpg,945,219,0.92,935,657,0.92,1184,766,0.81,676,756,0.8,0.178052,0.083365,0.094687
4,IMG_0017,IMG_0017_000005.jpg,947,218,0.92,937,653,0.91,1194,772,0.82,670,762,0.81,0.175422,0.082731,0.092691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4553,IMG_0026,IMG_0026_000565.jpg,896,191,0.93,896,664,0.93,1157,814,0.83,610,814,0.83,0.193848,0.083345,0.110504
4554,IMG_0026,IMG_0026_000566.jpg,894,191,0.94,894,665,0.93,1168,815,0.83,620,815,0.83,0.183388,0.082531,0.100857
4555,IMG_0026,IMG_0026_000567.jpg,895,192,0.94,895,666,0.93,1169,815,0.84,608,828,0.83,0.179982,0.082595,0.097387
4556,IMG_0026,IMG_0026_000568.jpg,902,191,0.94,890,663,0.93,1163,812,0.84,617,824,0.83,0.180179,0.082493,0.097687


In [9]:
df.to_excel("./result.xlsx")

keypoints": [
            "left_shoulder",
            "right_shoulder",
            "left_elbow",
            "right_elbow",
            "left_wrist",
            "right_wrist",
            "left_hip",
            "right_hip",
            "left_knee",
            "right_knee",
            "left_ankle",
            "right_ankle",
            "head",
            "neck"
            ]